# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.



**Appendage** Introduced the concept of KerasTuner to detect units ffor the dense layer


## Setup

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Prepare the data

In [ ]:
#Loading the dataset
num_classes = 10 #Softmax classification 
input_shape = (28, 28, 1)#Gray scale image and we have R*W as 28*28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# Scale images to the [0, 1] range
## This will help maintain consistency with the learning of weights and application of convolution on given 2-D grid topology; standardization is essential when we have only 1 learning weight
## Adittionally this tackles the problem of scaling which cannot be handled by convolution 

x_train = x_train.astype("float32") / 255
x_test = x_test.astype('float32') / 255

In [ ]:
# # Make sure images have shape (28, 28, 1)
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)

x_train=x_train.reshape(*x_train.shape,-1)
x_test=x_test.reshape(*x_test.shape,-1)

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# convert class vectors to binary class matrices
# y_train = 

classes=len(np.unique(y_train))
instances=y_train.shape[0]

temp=np.zeros(shape=(instances,classes))
temp[np.arange(len(y_train)),y_train]=1
np.concatenate([temp,y_train.reshape(-1,1)],axis=1)#Or we can use pd.get_dummies too
#to_categorical is a strong feature 

y_train= keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Build the model

In [ ]:
#This is a builder/constriuctor that appends one layer after the another 
# We can sequentially add these layers by using add to the sequential model or directly pass list of layers as an object


#Additional note kernel size is 3*3 used to find sparse local features, and we will have 32 such convolutional finders/kernels 
#Each convolution has a stride of 1 and thus the output shape will be (28-3+1)/1,(28-3+1)/1=26,26
#We will have 32*(3*3+1) weights--> 320== 32*9 +32 biases


#32 wala ek 28*28 pe lagaya gya thaa isliye 32 26*26 nikal ke aye


#max pooling will help introduce locational invariance and will further reduce the size of the topology 


#We will use one more layer of convolution to find association between pooled and accentuated features, example combination of rotated edges

#64 maps were applied to 32 maps
# kernel size was 3*3=9 therefore we have 64*32*9 +64 biases

#Every kernel comes with a bias

##Flatten everything to feed a sthe final feature

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

# 18496/64
# 32*9

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_1 (Dropout)         (None, 1600)             

## Train the model

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 13s 10ms/step - loss: 0.3633 - accuracy: 0.8919 - val_loss: 0.0844 - val_accuracy: 0.9760
Epoch 2/15
422/422 [==============================] - 4s 10ms/step - loss: 0.1099 - accuracy: 0.9662 - val_loss: 0.0585 - val_accuracy: 0.9838
Epoch 3/15
422/422 [==============================] - 4s 10ms/step - loss: 0.0862 - accuracy: 0.9734 - val_loss: 0.0470 - val_accuracy: 0.9862
Epoch 4/15
422/422 [==============================] - 4s 9ms/step - loss: 0.0699 - accuracy: 0.9781 - val_loss: 0.0414 - val_accuracy: 0.9893
Epoch 5/15
422/422 [==============================] - 4s 10ms/step - loss: 0.0620 - accuracy: 0.9804 - val_loss: 0.0364 - val_accuracy: 0.9895
Epoch 6/15
422/422 [==============================] - 4s 9ms/step - loss: 0.0551 - accuracy: 0.9827 - val_loss: 0.0380 - val_accuracy: 0.9898
Epoch 7/15
422/422 [==============================] - 4s 9ms/step - loss: 0.0503 - accuracy: 0.9844 - val_loss: 0.0335 - val_accuracy: 0.9910
E

## Evaluate the trained model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02424480766057968
Test accuracy: 0.9921000003814697


# Rough Work

In [ ]:
temp=np.zeros(shape=(instances,classes))
temp=temp[:3,:]

temp[:,0]=np.arange(1,4)*100
temp[np.arange(len(temp)),[0,1,0]]#-->[0,0],[1,1],[2,0]
# np.arange(len(temp))
# temp[[0,1,2],[0,1,0]]
# temp[:,[0,1,0,0,0]]#broadcasting

array([100.,   0., 300.])

In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import keras_tuner as kt
from tensorflow import keras

In [ ]:
from keras.layers.merge import concatenate
from keras.layers import *

https://keras.io/keras_tuner/

https://keras.io/guides/keras_tuner/getting_started/

In [ ]:
def search_model(hp):
  model = keras.Sequential(
      [
          keras.Input(shape=input_shape),
          layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2))])
  model.add(layers.Conv2D(
      hp.Choice('num_kernels',[16,32,64]),
      kernel_size=(3, 3), activation="relu"))
  
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes, activation="softmax"))

  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  return model




      
  

In [ ]:
tuner = kt.RandomSearch(
    search_model,
    objective='val_loss',
    max_trials=5)



In [ ]:
x_val=x_train[50000:,]
x_train=x_train[:50000,]

y_val=y_train[50000:,]
y_train=y_train[:50000,]



x_train.shape,x_val.shape,y_val.shape

((50000, 28, 28, 1), (10000, 28, 28, 1), (10000, 10))

In [ ]:
# tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))
best_model = tuner.get_best_models()[0]

In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0